In [1]:
# MPC Lab - Combined demo for:
# Part I: MPC Sum using additive secret sharing (3 parties)
# Part II: Shamir's Secret Sharing (k,n) with reconstruction (mod prime)
# Part III: Simple Garbled Circuit for AND gate (label-based, hash-derived encryption)
#
# Run in Google Colab as a single cell. Outputs are printed for screenshots.
import random
import hashlib
from functools import reduce

# -----------------------
# Part I: Additive secret sharing (3 parties) - MPC Sum
# -----------------------
print("=== Part I: MPC Sum using Additive Secret Sharing ===")
# Three parties: Ahmed, Ali, Rashida with private values whose sum = 220
private_values = {"Ahmed": 70, "Ali": 50, "Rashida": 100}
print("Private values (hidden):", {k: "***" for k in private_values})  # hide for demonstration

# Each party splits their value into 3 random shares that sum to the value (mod large int or plain int)
def split_into_shares(value, n_shares=3):
    shares = [random.randint(0, 1000) for _ in range(n_shares-1)]
    last = value - sum(shares)
    shares.append(last)
    return shares

# simulate distribution: each party creates shares and sends one share to each party (including self)
parties = list(private_values.keys())
n = len(parties)

# generate shares
all_shares = {p: split_into_shares(private_values[p], n) for p in parties}

# distribute: each party i will collect one share from every party j (share j->i is all_shares[j][i])
received = {p: [] for p in parties}
for j_idx, j in enumerate(parties):
    for i_idx, i in enumerate(parties):
        received[i].append(all_shares[j][i_idx])

# each party computes local sum of received shares
local_sums = {p: sum(received[p]) for p in parties}
print("Local sums (each party computed locally):", local_sums)

# global sum: sum of local sums should equal sum of private_values
global_sum = sum(local_sums.values())
expected_sum = sum(private_values.values())
print("Global sum (reconstructed):", global_sum)
print("Expected (true) sum:", expected_sum)
print("MPC Sum success:", global_sum == expected_sum)
print()

# For the report we print explicit shares (to include in screenshots) but normally shares are private
print("Shares (for debugging / report only):")
for p in parties:
    print(f"  {p} shares ->", all_shares[p])
print()

# -----------------------
# Part II: Shamir's Secret Sharing (k,n)
# -----------------------
print("=== Part II: Shamir's Secret Sharing (k,n) ===")
# We'll implement arithmetic over a prime field GF(p). Choose a prime > secret and > n.
def is_prime(n):
    if n < 2: return False
    if n % 2 == 0: return n == 2
    r = int(n**0.5)
    for i in range(3, r+1, 2):
        if n % i == 0:
            return False
    return True

# pick a prime for modulus (a reasonably small prime for demo)
PRIME = 2087  # > typical secret; 2087 is prime
assert is_prime(PRIME)

def mod_inv(a, p):
    # modular inverse via extended Euclid
    return pow(a, p-2, p)

def eval_poly(coeffs, x, p):
    # coeffs: [a0, a1, a2, ...] representing a0 + a1 x + a2 x^2 ...
    res = 0
    for power, c in enumerate(coeffs):
        res = (res + c * pow(x, power, p)) % p
    return res

def generate_shares(secret, k, n, p=PRIME):
    # random polynomial of degree k-1 with a0 = secret
    coeffs = [secret] + [random.randrange(0, p) for _ in range(k-1)]
    shares = [(i, eval_poly(coeffs, i, p)) for i in range(1, n+1)]
    return coeffs, shares

def lagrange_interpolate(x, x_s, y_s, p):
    # returns f(x) given points (x_s[i], y_s[i]) using Lagrange interpolation mod p
    total = 0
    k = len(x_s)
    for i in range(k):
        xi, yi = x_s[i], y_s[i]
        num, den = 1, 1
        for j in range(k):
            if i == j: continue
            xj = x_s[j]
            num = (num * (x - xj)) % p
            den = (den * (xi - xj)) % p
        inv_den = mod_inv(den, p)
        term = yi * num * inv_den
        total = (total + term) % p
    return total

# Demo parameters
secret = 1234
k = 3
n_shares = 5
coeffs, shares = generate_shares(secret, k, n_shares)
print("Secret:", secret)
print("Polynomial coeffs (mod p):", coeffs)
print("Shares (i, share_i):", shares)

# reconstruct using any k shares
selected = shares[1:k+1]  # pick shares indices 1..k
x_s = [s[0] for s in selected]
y_s = [s[1] for s in selected]
reconstructed = lagrange_interpolate(0, x_s, y_s, PRIME)  # f(0) = secret
print("Selected shares used for reconstruction:", selected)
print("Reconstructed secret:", reconstructed)
print("Reconstruction success:", reconstructed == secret)

# show failure if fewer than k shares (attempt using k-1 shares)
selected_few = shares[:k-1]
x_sf = [s[0] for s in selected_few]
y_sf = [s[1] for s in selected_few]
recon_few = lagrange_interpolate(0, x_sf, y_sf, PRIME)
print("Reconstruction with fewer than k shares (incorrect expected):", recon_few)
print()

# -----------------------
# Part III: Simple Garbled Circuit for AND gate
# -----------------------
print("=== Part III: Simple Garbled Circuit (AND gate demo) ===")
# We'll create random labels for each wire value 0/1, build a garbled table that maps input labels to output label
# Use keyed hash (SHA256) as "encryption" for label mixing.

def random_label():
    return random.getrandbits(128).to_bytes(16, byteorder='big')  # 16-byte label

def H(*args):
    h = hashlib.sha256()
    for a in args:
        if isinstance(a, int):
            h.update(str(a).encode())
        elif isinstance(a, bytes):
            h.update(a)
        else:
            h.update(str(a).encode())
    return h.digest()

# Create labels
wireA = {0: random_label(), 1: random_label()}
wireB = {0: random_label(), 1: random_label()}
wireOut = {0: random_label(), 1: random_label()}
gate_id = "AND_gate_1"

# Build garbled table: for each (a,b) store ciphertext = label_out XOR H(label_a || label_b || gate_id)
garbled_table = {}
for a in (0,1):
    for b in (0,1):
        La = wireA[a]
        Lb = wireB[b]
        Lout = wireOut[a & b]
        key = H(La, Lb, gate_id)
        # XOR Lout with key (truncate/expand key to label length)
        ct = bytes(x ^ y for x,y in zip(Lout, key[:len(Lout)]))
        garbled_table[(a,b)] = ct

# Evaluator: given actual input bits and their labels, decrypt
def evaluate_and(label_a, label_b):
    # compute key and find label_out by XOR with stored ct corresponding to the (a,b) pair
    key = H(label_a, label_b, gate_id)
    ct = garbled_table[(None,None)] if False else None  # placeholder
    # Instead, find correct table entry by trying all 4 entries: we simulate evaluator knowing label_a and label_b only
    # The evaluator will compute key and attempt to decrypt each entry, but normally the evaluator is given only one ct via OT.
    for (a,b), ct in garbled_table.items():
        # compute expected key from provided labels
        if label_a == wireA[a] and label_b == wireB[b]:
            Lout = bytes(x ^ y for x,y in zip(ct, key[:len(ct)]))
            # match Lout to wireOut labels to get bit value
            for bit, lab in wireOut.items():
                if Lout == lab:
                    return bit, Lout
    return None, None

# Simulate: Omar has A, Nancy has B
A = 1  # Omar's bit
B = 0  # Nancy's bit
label_A = wireA[A]
label_B = wireB[B]

# In a real protocol, labels are exchanged via Oblivious Transfer and evaluator receives only the correct label for each input.
out_bit, out_label = evaluate_and(label_A, label_B)
print(f"Omar's input A = {A}, Nancy's input B = {B}")
print("Evaluated AND result (securely):", out_bit)
print("Expected AND:", A & B)
print("Garbled circuit evaluation success:", out_bit == (A & B))

# Print garbled table entries lengths (for screenshot)
print("Garbled table entries (hex prefixes):")
for k, ct in garbled_table.items():
    print(f"  {k}: {ct.hex()[:24]}...")
print()

# -----------------------
# End of demo
# -----------------------
print("=== End of MPC Lab demo ===")


=== Part I: MPC Sum using Additive Secret Sharing ===
Private values (hidden): {'Ahmed': '***', 'Ali': '***', 'Rashida': '***'}
Local sums (each party computed locally): {'Ahmed': 1086, 'Ali': 888, 'Rashida': -1754}
Global sum (reconstructed): 220
Expected (true) sum: 220
MPC Sum success: True

Shares (for debugging / report only):
  Ahmed shares -> [13, 471, -414]
  Ali shares -> [80, 40, -70]
  Rashida shares -> [993, 377, -1270]

=== Part II: Shamir's Secret Sharing (k,n) ===
Secret: 1234
Polynomial coeffs (mod p): [1234, 461, 730]
Shares (i, share_i): [(1, 338), (2, 902), (3, 839), (4, 149), (5, 919)]
Selected shares used for reconstruction: [(2, 902), (3, 839), (4, 149)]
Reconstructed secret: 1234
Reconstruction success: True
Reconstruction with fewer than k shares (incorrect expected): 1861

=== Part III: Simple Garbled Circuit (AND gate demo) ===
Omar's input A = 1, Nancy's input B = 0
Evaluated AND result (securely): 0
Expected AND: 0
Garbled circuit evaluation success: True
Ga